In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv

load_dotenv()

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

client = OpenAI()

In [2]:
assistant = client.beta.assistants.create(
    name='GIS Assistant',
    instructions='You are a GIS assistant, and will perform and answer questions about geospatial analysis.',
    model='gpt-4-1106-preview',
)

show_json(assistant)

{'id': 'asst_LqOdwmpQvZcZawke8ku3brDJ',
 'created_at': 1699950334,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a GIS assistant, and will perform and answer questions about geospatial analysis.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'GIS Assistant',
 'object': 'assistant',
 'tools': []}

In [3]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_twRPzA5QcmClxKWvRmLzDCtH',
 'created_at': 1699950334,
 'metadata': {},
 'object': 'thread'}

In [4]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='What is the rate of deforestation in Norway?'
)

show_json(message)

{'id': 'msg_c6fDZStp2jzYpOMlz55QBaYo',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'What is the rate of deforestation in Norway?'},
   'type': 'text'}],
 'created_at': 1699950334,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_twRPzA5QcmClxKWvRmLzDCtH'}

Respond to user question

In [5]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

show_json(run)

{'id': 'run_rUwnYIsz50YqUPuNTyF3vVno',
 'assistant_id': 'asst_LqOdwmpQvZcZawke8ku3brDJ',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1699950335,
 'expires_at': 1699950935,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a GIS assistant, and will perform and answer questions about geospatial analysis.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_twRPzA5QcmClxKWvRmLzDCtH',
 'tools': []}

In [6]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [7]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_rUwnYIsz50YqUPuNTyF3vVno',
 'assistant_id': 'asst_LqOdwmpQvZcZawke8ku3brDJ',
 'cancelled_at': None,
 'completed_at': 1699950343,
 'created_at': 1699950335,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a GIS assistant, and will perform and answer questions about geospatial analysis.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1699950335,
 'status': 'completed',
 'thread_id': 'thread_twRPzA5QcmClxKWvRmLzDCtH',
 'tools': []}

In [8]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_OVagaakhwbwcgvv3Y0QnZHUx',
   'assistant_id': 'asst_LqOdwmpQvZcZawke8ku3brDJ',
   'content': [{'text': {'annotations': [],
      'value': 'As of my last update in April 2023, I do not have real-time data or access to databases that can provide the current rate of deforestation in Norway. However, Norway has traditionally had very low rates of deforestation due to stringent environmental policies, a strong economy that does not rely heavily on logging, and forest management practices that are designed to be sustainable.\n\nIn recent years, the country has even been known to experience a net increase in forest area thanks to these effective management practices and reforestation efforts. Norway has also taken a strong stance globally by funding anti-deforestation projects in other countries, as part of its commitment to environmental conservation.\n\nFor the most current deforestation rates in Norway, you would typically reference data from sources such as the Norwe

In [9]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role='user', content='How about Sweden?'
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_6zGaVuZbOE5hWXIO4JOnAy71',
   'assistant_id': 'asst_LqOdwmpQvZcZawke8ku3brDJ',
   'content': [{'text': {'annotations': [],
      'value': 'Similar to Norway, Sweden has a reputation for sustainably managing its forest resources. Sweden is one of Europe\'s largest producers of wood, and forestry is an important part of its economy. Despite this, the country has strict environmental laws and sustainable forestry practices that balance economic interests with conservation.\n\nSweden\'s approach to forestry includes continuous tree planting and a legal requirement for replanting after felling, which ensures that the rate of deforestation is kept low and that there is a net increase in forest growth over time rather than a decrease. The use of "clear-cutting" is balanced by these reforestation practices, and the overall growth of forests in Sweden has historically been positive.\n\nJust like with Norway, for the most recent and accurate data on Sweden\'s deforestation 